<a href="https://colab.research.google.com/github/Ayushee-Seeburrun/ASAG-with-Data-Augmentation/blob/main/solving_duplicates.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd

In [25]:
finaldataset = pd.read_csv("/content/drive/MyDrive/Data Augmentation/Dataset Splits/train_balanced_s2_s3_final.csv")
print("rows before removing duplicate score-3 rows: ", len(finaldataset))
print(finaldataset["Score2"].value_counts().sort_index())

rows before removing duplicate score-3 rows:  18247
Score2
0    5424
1    4518
2    4225
3    4080
Name: count, dtype: int64


In [42]:
finaldataset_3 = finaldataset[finaldataset["Score2"].isin([3])]

total_withdupes = finaldataset_3[finaldataset_3.duplicated(subset=["Score2", "EssayText"], keep=False)]

duplicates_3 = finaldataset_3.duplicated(subset=["Score2", "EssayText"], keep="first").sum()
print("Total rows including their duplicates(in Score 3): ", len(total_withdupes))
print("Duplicates to be removed for score 3 : ", duplicates_3)

Total rows including their duplicates(in Score 3):  1982
Duplicates to be removed for score 3 :  991


In [10]:
import re, unicodedata

def clean_text(s):
    s = str(s)
    s = unicodedata.normalize("NFKC", s)
    s = s.replace("\u00A0", " ")
    s = re.sub(r"\s+", " ", s).strip()
    return s

finaldataset["EssayText"] = finaldataset["EssayText"].map(clean_text)

# Split
original = finaldataset[finaldataset["type"] != "paraphrased"]     # keep all originals
para = finaldataset[finaldataset["type"] == "paraphrased"]

# Remove duplicates only inside paraphrased rows and not dropping the original rows
para = para.drop_duplicates(subset=["Score2", "EssayText"])

# Remove paraphrases versions that are exactly the same as an original row(a duplicate of the original row)
orig_keys = set(zip(original["Score2"], original["EssayText"]))
para = para[~para.apply(lambda r: (r["Score2"], r["EssayText"]) in orig_keys, axis=1)]

finaldataset = pd.concat([original, para], ignore_index=True)


In [12]:
finaldataset.to_csv("/content/drive/MyDrive/Data Augmentation/Dataset Splits/train_balanced_final.csv",index=False)

In [46]:
fd = pd.read_csv("/content/drive/MyDrive/Data Augmentation/Dataset Splits/train_balanced_final.csv")
print("rows after removing duplicate rows: ")
print(fd["Score2"].value_counts().sort_index())

rows after removing duplicate rows: 
Score2
0    5424
1    4518
2    4221
3    3089
Name: count, dtype: int64


In [47]:
print(len(pd.read_csv("/content/drive/MyDrive/Data Augmentation/Dataset Splits/train_set.csv")))

13765


In [32]:
fd_3 = fd[fd["Score2"].isin([3])]

duplicates_3 = fd_3.duplicated(subset=["Score2", "EssayText"], keep=False).sum()
print("Duplicates for score 3: ",duplicates_3)


Duplicates for score 3:  0
